<a id="top_of_page">

# Working with Tabular Data

## Table of Contents

- [CLI tool to view sample of CSV file](#cli)
- [Read CSV file](#read_csv)
- [Viewing 3 Columns at a Time](#view_n_columns_at_a_time)
- [Getting rid of columns using the drop() method](#drop_column)
- [Extracting parts of a string with `substr()`](#substr)
- [Creating new column with withColumn()](#withColumn)
- [Renaming one column at a type, the withColumnRenamed() way](#withColumnRenamed)
- [Changing multiple columns - Batch lowercasing using the toDF() method](#toDF)
- [Selecting our columns in alphabetical order using select()](#sorting_columns)
- [Diagnosing a dataframe with describe() and summary()](#describe_and_summarize)
- [Joining Data](#joining)
- [INNER join](#inner_join)

<a id="cli">

## Useful CLI tool to initially assess or view sample of CSV file

[[back to top]](#top_of_page)

Linux Command Line:
- `head -n 5 BroadcastLogs_2018_Q3_M8_sample.csv`    # To view first 5 rows of the csv file


Windows Command Line:
- Not really sure what CLI tool can be used??  Perhaps just use pandas read_csv() with nrows= parameter?

<a id="read_csv">

## Read CSV file

[[back to top]](#top_of_page)

In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
import os
import pyspark.sql.functions as F

In [2]:
spark = (
    SparkSession.builder
    # Enable eager, interactive mode - typically do not do this with production code
    .config("spark.sql.repl.eagerEval.enabled", "True")
    .config("spark.executor.cores", str(os.cpu_count()))
    .appName("Working with Tabular Data")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

In [3]:
DIRECTORY = Path("./data/broadcast_logs")

In [4]:
logs = spark.read.csv(
    str(DIRECTORY / "BroadcastLogs_2018_Q3_M8.CSV"),
    sep="|",
    header=True,
    inferSchema=True,
    timestampFormat="yyyy-MM-dd",
)

In [5]:
logs.printSchema()

root
 |-- BroadcastLogID: integer (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- SequenceNO: integer (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string 

<a id="view_n_columns_at_a_time">

## Viewing 3 Columns at a Time

[[back to top]](#top_of_page)

In [6]:
import numpy as np

In [7]:
column_split = np.array_split(
    np.array(logs.columns), len(logs.columns) // 3
)

In [8]:
print(column_split)

[array(['BroadcastLogID', 'LogServiceID', 'LogDate'], dtype='<U22'), array(['SequenceNO', 'AudienceTargetAgeID', 'AudienceTargetEthnicID'],
      dtype='<U22'), array(['CategoryID', 'ClosedCaptionID', 'CountryOfOriginID'], dtype='<U22'), array(['DubDramaCreditID', 'EthnicProgramID', 'ProductionSourceID'],
      dtype='<U22'), array(['ProgramClassID', 'FilmClassificationID', 'ExhibitionID'],
      dtype='<U22'), array(['Duration', 'EndTime', 'LogEntryDate'], dtype='<U22'), array(['ProductionNO', 'ProgramTitle', 'StartTime'], dtype='<U22'), array(['Subtitle', 'NetworkAffiliationID', 'SpecialAttentionID'],
      dtype='<U22'), array(['BroadcastOriginPointID', 'CompositionID', 'Producer1'],
      dtype='<U22'), array(['Producer2', 'Language1', 'Language2'], dtype='<U22')]


In [9]:
for x in column_split:
    logs.select(*x).show(5, False)

+--------------+------------+-------------------+
|BroadcastLogID|LogServiceID|LogDate            |
+--------------+------------+-------------------+
|1196192316    |3157        |2018-08-01 00:00:00|
|1196192317    |3157        |2018-08-01 00:00:00|
|1196192318    |3157        |2018-08-01 00:00:00|
|1196192319    |3157        |2018-08-01 00:00:00|
|1196192320    |3157        |2018-08-01 00:00:00|
+--------------+------------+-------------------+
only showing top 5 rows

+----------+-------------------+----------------------+
|SequenceNO|AudienceTargetAgeID|AudienceTargetEthnicID|
+----------+-------------------+----------------------+
|1         |4                  |null                  |
|2         |null               |null                  |
|3         |null               |null                  |
|4         |null               |null                  |
|5         |null               |null                  |
+----------+-------------------+----------------------+
only showing top 5 ro

<a id="drop_column">

## Getting rid of columns using the drop() method

[[back to top]](#top_of_page)

In [10]:
logs = logs.drop("BroadcastLogID", "SequenceNO")
 
# Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs.columns)  # => False
print("SequenceNo" in logs.columns)  # => False

False
False


#### Another way using select() with list comprehension and then unpacking items in list to strings of column names

In [ ]:
logs = logs.select(
    *[x for x in logs.columns if x not in ["BroadcastLogID", "SequenceNO"]]
)

# Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs.columns)  # => False
print("SequenceNo" in logs.columns)  # => False

## Unfortunate Inconsistency When Unpacking is Needed or Not

With `select()`, you don't necessarily need to unpack list, but with `drop()`, you do need to explicitly unpack a list.  Therefore, best practice to always unpack/* a list.

<a id="substr">

## Extracting parts of a string with `substr()`

[[back to top]](#top_of_page)

In [11]:
logs.select(F.col("Duration")).show(5)

+----------------+
|        Duration|
+----------------+
|02:00:00.0000000|
|00:00:30.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
|00:00:15.0000000|
+----------------+
only showing top 5 rows



#### What data type is "Duration" column?

In [12]:
print(logs.select(F.col("Duration")).dtypes)

[('Duration', 'string')]


[REFERENCE](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html) - Datetime patterns for formatting and parsing

In [13]:
logs.select(
    F.col("Duration"),
    F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"),
    F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"),
    F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds"),
).distinct().show(5)

+----------------+---------+-----------+-----------+
|        Duration|dur_hours|dur_minutes|dur_seconds|
+----------------+---------+-----------+-----------+
|00:10:06.0000000|        0|         10|          6|
|00:10:37.0000000|        0|         10|         37|
|00:04:52.0000000|        0|          4|         52|
|00:26:41.0000000|        0|         26|         41|
|00:08:18.0000000|        0|          8|         18|
+----------------+---------+-----------+-----------+
only showing top 5 rows



In [14]:
logs.select(
    F.col("Duration"),
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ).alias("Duration_seconds"),
).distinct().show(5)

+----------------+----------------+
|        Duration|Duration_seconds|
+----------------+----------------+
|00:10:30.0000000|             630|
|00:25:52.0000000|            1552|
|00:28:08.0000000|            1688|
|06:00:00.0000000|           21600|
|00:32:08.0000000|            1928|
+----------------+----------------+
only showing top 5 rows



<a id="withColumn">

## Creating new column with withColumn()

[[back to top]](#top_of_page)

In [15]:
logs = logs.withColumn(
    "Duration_seconds",
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ),
)

In [16]:
logs.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttenti

<a id="withColumnRenamed">

## Renaming one column at a type, the withColumnRenamed() way

[[back to top]](#top_of_page)

In [17]:
logs = logs.withColumnRenamed("Duration_seconds", "duration_seconds")

In [18]:
logs.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttenti

<a id="toDF">

## Changing multiple columns - Batch lowercasing using the toDF() method

[[back to top]](#top_of_page)

In [19]:
logs.toDF(*[x.lower() for x in logs.columns]).printSchema()

root
 |-- logserviceid: integer (nullable = true)
 |-- logdate: timestamp (nullable = true)
 |-- audiencetargetageid: integer (nullable = true)
 |-- audiencetargetethnicid: integer (nullable = true)
 |-- categoryid: integer (nullable = true)
 |-- closedcaptionid: integer (nullable = true)
 |-- countryoforiginid: integer (nullable = true)
 |-- dubdramacreditid: integer (nullable = true)
 |-- ethnicprogramid: integer (nullable = true)
 |-- productionsourceid: integer (nullable = true)
 |-- programclassid: integer (nullable = true)
 |-- filmclassificationid: integer (nullable = true)
 |-- exhibitionid: integer (nullable = true)
 |-- duration: string (nullable = true)
 |-- endtime: string (nullable = true)
 |-- logentrydate: timestamp (nullable = true)
 |-- productionno: string (nullable = true)
 |-- programtitle: string (nullable = true)
 |-- starttime: string (nullable = true)
 |-- subtitle: string (nullable = true)
 |-- networkaffiliationid: integer (nullable = true)
 |-- specialattenti

<a id="sorting_columns">

## Selecting our columns in alphabetical order using select()

[[back to top]](#top_of_page)

In [20]:
logs.select(sorted(logs.columns)).printSchema()

root
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- BroadcastOriginPointID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CompositionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- Language1: integer (nullable = true)
 |-- Language2: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- Producer1: string (nullable = true)
 |-- Producer2: string (nullable = true)
 |-- ProductionNO: 

<a id="describe_and_summarize">

## Diagnosing a dataframe with describe() and summary()

[[back to top]](#top_of_page)

In [21]:
for i in logs.columns:
    logs.describe(i).show()

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|           7169318|
|   mean|3453.8804215407936|
| stddev|200.44137201584468|
|    min|              3157|
|    max|              3925|
+-------+------------------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+

+-------+-------------------+
|summary|AudienceTargetAgeID|
+-------+-------------------+
|  count|             482674|
|   mean| 3.4819298325577925|
| stddev| 1.0446667834126653|
|    min|                  1|
|    max|                  4|
+-------+-------------------+

+-------+----------------------+
|summary|AudienceTargetEthnicID|
+-------+----------------------+
|  count|                 51613|
|   mean|    123.54631585065778|
| stddev|      75.2748950313355|
|    min|                     4|
|    max|                   337|
+-------+----------------------+

+-------+-----------------+
|summary|       CategoryID|
+-------+-------------

In [23]:
for i in logs.columns:
    logs.select(i).summary().show()

+-------+------------------+
|summary|      LogServiceID|
+-------+------------------+
|  count|           7169318|
|   mean|3453.8804215407936|
| stddev|200.44137201584468|
|    min|              3157|
|    25%|              3291|
|    50%|              3384|
|    75%|              3628|
|    max|              3925|
+-------+------------------+

+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    25%|
|    50%|
|    75%|
|    max|
+-------+

+-------+-------------------+
|summary|AudienceTargetAgeID|
+-------+-------------------+
|  count|             482674|
|   mean| 3.4819298325577925|
| stddev| 1.0446667834126653|
|    min|                  1|
|    25%|                  4|
|    50%|                  4|
|    75%|                  4|
|    max|                  4|
+-------+-------------------+

+-------+----------------------+
|summary|AudienceTargetEthnicID|
+-------+----------------------+
|  count|                 51613|
|   mean|    123.54631585065778|
| st

<a id="joining">

## Joining Data

[[back to top]](#top_of_page)

#### Let's create a 2nd dataframe called "log_identifier"

In [38]:
DIRECTORY = Path("./data/broadcast_logs")
log_identifier = spark.read.csv(
    str(DIRECTORY / "ReferenceTables/LogIdentifier.csv"),
    sep="|",
    header=True,
    inferSchema=True,
)

In [39]:
log_identifier.printSchema()

root
 |-- LogIdentifierID: string (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- PrimaryFG: integer (nullable = true)



In [40]:
log_identifier.show(10)

+---------------+------------+---------+
|LogIdentifierID|LogServiceID|PrimaryFG|
+---------------+------------+---------+
|           13ST|        3157|        1|
|         2000SM|        3466|        1|
|           70SM|        3883|        1|
|           80SM|        3590|        1|
|           90SM|        3470|        1|
|         9DAPTN|        3158|        1|
|         9DCFCF|        3159|        1|
|         9DCFRN|        3160|        1|
|         9DCHRO|        3161|        1|
|         9DCIVI|        3162|        1|
+---------------+------------+---------+
only showing top 10 rows



In [41]:
log_identifier.select(F.col("PrimaryFG")).distinct()

PrimaryFG
1
0


#### Let's do a simple groupBy count

In [44]:
(
    log_identifier
    .select(F.col("PrimaryFG"))
    .groupBy(F.col("PrimaryFG"))
    .count()
)

PrimaryFG,count
1,758
0,162


In [45]:
log_identifier.count()

920

#### Filter or limit our log_identifier to have PrimaryFG equal to 1

In [46]:
log_identifier = log_identifier.where(F.col("PrimaryFG") == 1)
print(log_identifier.count())

758


#### Let's compare or see which column we can join `logs` and `log_identifier` with.  Let's print out the schema for both:

In [47]:
logs.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttenti

In [48]:
log_identifier.printSchema()

root
 |-- LogIdentifierID: string (nullable = true)
 |-- LogServiceID: integer (nullable = true)
 |-- PrimaryFG: integer (nullable = true)



#### From the data dictionary and also based on the columns listed, looks like we can join with the `LogServiceID` column

<a id="inner_join">

## Perform INNER join

[[back to top]](#top_of_page)

In [50]:
logs_and_channels = logs.join(
    log_identifier,
    on="LogServiceID",
    how="inner"
)

In [51]:
logs_and_channels.printSchema()

root
 |-- LogServiceID: integer (nullable = true)
 |-- LogDate: timestamp (nullable = true)
 |-- AudienceTargetAgeID: integer (nullable = true)
 |-- AudienceTargetEthnicID: integer (nullable = true)
 |-- CategoryID: integer (nullable = true)
 |-- ClosedCaptionID: integer (nullable = true)
 |-- CountryOfOriginID: integer (nullable = true)
 |-- DubDramaCreditID: integer (nullable = true)
 |-- EthnicProgramID: integer (nullable = true)
 |-- ProductionSourceID: integer (nullable = true)
 |-- ProgramClassID: integer (nullable = true)
 |-- FilmClassificationID: integer (nullable = true)
 |-- ExhibitionID: integer (nullable = true)
 |-- Duration: string (nullable = true)
 |-- EndTime: string (nullable = true)
 |-- LogEntryDate: timestamp (nullable = true)
 |-- ProductionNO: string (nullable = true)
 |-- ProgramTitle: string (nullable = true)
 |-- StartTime: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- NetworkAffiliationID: integer (nullable = true)
 |-- SpecialAttenti